In [1]:
from collections import defaultdict, Counter
import numpy as np

In [2]:
fn = r"train 2.tsv"
handle  = open(fn, "r")
tokens = list()

for line in handle:
  entry = line.strip().split("\t")
  tokens.append( ( entry[0], entry[1]) )

In [3]:
#frequencies of all the mutations
frequency = Counter([t for (w,t) in tokens])
frequency

Counter({'N': 8584022, 'S': 973469, 'U': 327110, 'H': 80504, 'T': 34895})

In [4]:
def flat2sent(sents, end = "<S>"):
  list_of_sent = list()
  sent = []
  for char in sents:
    sent.append(char)
    if char[0] == end:
      list_of_sent.append(sent)
      sent = []
  return list_of_sent


In [5]:
#train-split for sentences
train_index = int([ind for (ind, token) in enumerate(tokens) if token[0] == "<S>"][-1]*0.8)
sentTrain = flat2sent ( tokens[:train_index] )
sentTest = flat2sent( tokens[train_index:] )

In [6]:
max(len( sent ) for sent in sentTrain)

147

In [19]:
vocabs = set(piece[0] for piece in tokens[:train_index])
word2index = {w:i for i,w in enumerate(vocabs)}
mutate2index = {m:i for i,m in enumerate(["N","S","U","H","T"])}
trainX = [[word2index[w[0]] for w in s] for s in sentTrain]
#add padding here
trainY = [[mutate2index[w[1]] for w in s] for s in sentTrain]

In [20]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Embedding, Dense, Dropout

model = Sequential()
model.add(Embedding(input_dim=len(vocabs), output_dim=32, input_length=50))
model.add(LSTM(units=64, recurrent_dropout=0.1))
model.add(Dense(5, activation="softmax"))
model.add(Dense(1, activation="softmax"))
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 50, 32)            4828960   
                                                                 
 lstm_3 (LSTM)               (None, 64)                24832     
                                                                 
 dense_4 (Dense)             (None, 5)                 325       
                                                                 
 dense_5 (Dense)             (None, 1)                 6         
                                                                 
Total params: 4,854,123
Trainable params: 4,854,123
Non-trainable params: 0
_________________________________________________________________
None


In [36]:
#import numpy as np
np.asarray(trainX, dtype=np.int32)
#model.fit(trainX[0], trainY[0])
#max(len(i) for i in trainX )
#[print(i.shape, i.dtype) for i in model.inputs]

ValueError: ignored

In [ ]:
# argument sent is a list of [token,label] pairs; return number of correctly predicted labels
def predict_from_scratch(sent, model=None):
  correct = 0
  for token in sent:
    guess = random.choice(['S','U','T','H','N'])
    if guess == token[1]:
      correct += 1
  return correct

In [ ]:
# argument sent is a list of [token,label] pairs; return number of correctly predicted labels
def predict_anything_goes(sent):
  correct = 0
  for token in sent:
    guess = 'N'
    if guess == token[1]:
      correct += 1
  return correct

In [ ]:
#sentence is a list of tuples(x,y)
def evaluate():
  total = 0
  correct_from_scratch = 0
  correct_anything_goes = 0
  testfile = open('test.tsv', 'r')
  sentence = []
  for line in testfile:
    total += 1
    pieces = line.rstrip("\n").split("\t")
    if pieces[0]=='<S>':
      correct_from_scratch += predict_from_scratch(sentence)
      correct_anything_goes += predict_anything_goes(sentence)
      sentence = []
    else:
      sentence.append(pieces)
  correct_from_scratch += predict_from_scratch(sentence)
  correct_anything_goes += predict_anything_goes(sentence)
  return (correct_from_scratch/total, correct_anything_goes/total)

In [ ]:
evaluate()

(0.191809, 0.819613)